In [1]:
import numpy as np
import tensorflow as tf
import time
# import random

2022-01-06 10:27:06.770114: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
gpu_available = tf.test.is_gpu_available()
is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
is_cuda_gpu_min_3 = tf.test.is_gpu_available(True, (3,0))
is_cuda_gpu_min_3

In [2]:
train = np.load('../data/freebase15k_numpy/train.npy')
validation = np.load('../data/freebase15k_numpy/validation.npy')
test = np.load('../data/freebase15k_numpy/test.npy')

entities = np.load('../data/freebase15k_numpy/entities.npy')
relations = np.load('../data/freebase15k_numpy/relations.npy')

train = tf.convert_to_tensor(train.astype(dtype=np.int16), dtype=tf.int32)
validation = tf.convert_to_tensor(validation.astype(dtype=np.int16), dtype=tf.int32)
test = tf.convert_to_tensor(test.astype(dtype=np.int16), dtype=tf.int32)
relations = tf.convert_to_tensor(relations.astype(dtype=np.int16), dtype=tf.int16)
entities = tf.convert_to_tensor(entities.astype(dtype=np.float64), dtype=tf.float64)

2022-01-06 10:27:10.032494: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-06 10:27:10.033328: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-06 10:27:10.061850: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-06 10:27:10.062012: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1060 computeCapability: 6.1
coreClock: 1.6705GHz coreCount: 10 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 178.99GiB/s
2022-01-06 10:27:10.062056: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-06 10:27:10.063576: I tensorflow/stream_executor/platform/d

In [3]:
iteration = tf.constant(500)
dim = tf.constant(100)
kns = tf.constant(5)
alpha = tf.Variable(0.000001, dtype = tf.float64)
beta = tf.Variable(0.0000001, dtype = tf.float64)
z = tf.constant(0.5, dtype = tf.float64)
nn1 = tf.random.uniform( shape=(entities.shape[0], dim),
      minval=tf.math.truediv(z , tf.cast(dim,tf.float64),'minval'), 
      maxval=tf.math.truediv(z , tf.cast(dim,tf.float64)),
      dtype=tf.dtypes.float64, seed=None, name='nn1')
      
nn0 =tf.random.uniform(shape=(entities.shape[0], dim), 
      minval=tf.math.truediv(z , tf.cast(dim,tf.float64)), 
      maxval=tf.math.truediv(z , tf.cast(dim,tf.float64)),
      dtype=tf.dtypes.float64, seed=None, name='nn0')
nn2 = tf.random.uniform(shape=(relations.shape[0], dim), 
      minval=tf.math.truediv(z , tf.cast(dim,tf.float64)), 
      maxval=tf.math.truediv(z , tf.cast(dim,tf.float64)),
      dtype=tf.dtypes.float64, seed=None, name='nn2')

In [96]:
def calculateRank(triple):   
    selectedEntities = tf.cast(tf.gather(entities, 0, axis=1), dtype=tf.int32)
    
    _head_index = tf.gather(triple,0)
    _tail_index = tf.gather(triple,1)
    _relation_index = tf.gather(triple,2)  

    indexes = tf.where( tf.math.logical_and( (tf.gather(train, 1, axis=1) == _tail_index) , (tf.gather(train, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(train, indexes), 0, axis=1)
    shape = tf.constant([entities.shape[0]])
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesTrain = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 


    indexes = tf.where( tf.math.logical_and( (tf.gather(test, 1, axis=1) == _tail_index) , (tf.gather(test, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(test, indexes), 0, axis=1)
    shape = tf.constant([entities.shape[0]])
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesTest = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 
    
    indexes = tf.where( tf.math.logical_and( (tf.gather(validation, 1, axis=1) == _tail_index) , (tf.gather(validation, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(validation, indexes), 0, axis=1)
    shape = tf.constant([entities.shape[0]])
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesvalidation = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 

    selectedEntitiesTrain= tf.reshape(selectedEntitiesTrain, (1, selectedEntitiesTrain.shape[0]))
    selectedEntitiesTest = tf.reshape(selectedEntitiesTest, (1, selectedEntitiesTest.shape[0]))
    selectedEntitiesvalidation = tf.reshape(selectedEntitiesvalidation, (1, selectedEntitiesvalidation.shape[0]))
    selectedEntitiesFinal = tf.sets.intersection(selectedEntitiesTrain, selectedEntitiesTest)
    selectedEntitiesFinal = tf.cast(tf.sets.intersection(selectedEntitiesFinal, selectedEntitiesvalidation), dtype=tf.int32).values
    couraptedH = tf.concat([  tf.reshape(selectedEntitiesFinal, (selectedEntitiesFinal.shape[0], 1)) , tf.fill([selectedEntitiesFinal.shape[0], 1], _tail_index) , tf.fill([selectedEntitiesFinal.shape[0], 1], _relation_index)],1)
    couraptedH = tf.concat([couraptedH, tf.reshape(triple, (1,3))], 0)
    p =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(tf.gather(nn1,tf.gather(couraptedH,1, axis=1),0),tf.transpose(  tf.gather( nn0,  tf.gather( couraptedH,0, axis=1)) + tf.gather(nn2,_relation_index) ),axes=1 )))
    rankH = tf.where(tf.math.top_k( p ,k = p.shape[0]).indices  == tf.subtract(p.shape[0] , 1) )
    couraptedH = None
    indexes = None
    existEntites = None
    selectedEntitiesTrain = None
    selectedEntitiesTest = None
    selectedEntitiesvalidation = None
    selectedEntitiesFinal = None
    p = None

    indexes = tf.where( tf.math.logical_and( (tf.gather(train, 0, axis=1) == _head_index) , (tf.gather(train, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(train, indexes), 1, axis=1)
    shape = tf.constant([entities.shape[0]])
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesTrain = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 
    indexes = tf.where( tf.math.logical_and( (tf.gather(test, 0, axis=1) == _head_index) , (tf.gather(test, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(test, indexes), 1, axis=1)
    shape = tf.constant([entities.shape[0]])
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesTest = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 
    indexes = tf.where( tf.math.logical_and( (tf.gather(validation, 0, axis=1) == _head_index) , (tf.gather(validation, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(validation, indexes), 1, axis=1)
    shape = tf.constant([entities.shape[0]])
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesvalidation = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 
    selectedEntitiesTrain= tf.reshape(selectedEntitiesTrain, (1, selectedEntitiesTrain.shape[0]))
    selectedEntitiesTest = tf.reshape(selectedEntitiesTest, (1, selectedEntitiesTest.shape[0]))
    selectedEntitiesvalidation = tf.reshape(selectedEntitiesvalidation, (1, selectedEntitiesvalidation.shape[0]))

    selectedEntitiesFinal = tf.sets.intersection(selectedEntitiesTrain, selectedEntitiesTest)
    selectedEntitiesFinal = tf.cast(tf.sets.intersection(selectedEntitiesFinal, selectedEntitiesvalidation), dtype=tf.int32).values
    couraptedT = tf.concat([   tf.fill([selectedEntitiesFinal.shape[0], 1], _head_index) , tf.reshape(selectedEntitiesFinal, (selectedEntitiesFinal.shape[0], 1)) , tf.fill([selectedEntitiesFinal.shape[0], 1], _relation_index)],1)
    couraptedT = tf.concat([couraptedT, tf.reshape(triple, (1,3))], 0)
    p =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(tf.gather(nn1,tf.gather(couraptedT,1, axis=1),0),tf.transpose(  tf.gather( nn0,  tf.gather( couraptedT,0, axis=1)) + tf.gather(nn2,_relation_index) ),axes=1 )))
    rankT = tf.where(tf.math.top_k( p ,k = p.shape[0]).indices  == tf.subtract(p.shape[0] , 1) )
    print(rankH, rankT)
    if tf.math.greater(rankH , rankT):
        return rankT
    return rankH

In [ ]:
def mrr(ranks):
    inverse = []
    for rank in ranks:
        inverse.append(float(1)/rank)
    summ = sum(inverse)
    return float(1)/len(inverse) * summ

# 483142    

In [ ]:
start = time.time()
paddings = tf.constant([[0, 1,], [0, 0]])
t_row = train.shape[0]
for x in range(0 , iteration):
    # tf.random.shuffle(train, seed=None, name=None)
    with tf.device('/cpu:0'):
        total_samples  = tf.gather( tf.random.categorical([tf.gather(entities, 2, axis=1)], tf.math.multiply(kns,t_row) , dtype=None, seed=None),0)
    for tIndex in range(0 , t_row):
        triple = tf.gather(train, tIndex)
        _head_index = tf.gather(triple,0)
        _relation_index = tf.gather(triple,2)
        _vHead = tf.gather(nn0,_head_index)
        _vRel = tf.gather(nn2,_relation_index)

        samples = [tf.slice(total_samples,begin=[tf.math.multiply(tIndex, kns)],size=[kns])]
        samples = tf.pad(samples, paddings, constant_values=0)
        samples =  tf.cast(samples, tf.int64)         
        samples = tf.transpose(tf.concat([samples, [[tf.gather(triple,1)],[1]]], 1))
        indices = tf.gather(samples, 0, axis=1)
        _nn1_samples = tf.gather(nn1, indices)
        _sigmoid =tf.math.sigmoid(tf.tensordot( _nn1_samples , tf.transpose(tf.math.add(_vHead , _vRel)) , axes=1))
        cost = tf.math.subtract(tf.cast(tf.gather(samples, 1, axis=1), tf.float64) , _sigmoid)
        g = tf.math.multiply(alpha , cost)
        g1 = tf.math.multiply(beta , cost)
        _nn1_samples = tf.math.add(tf.math.multiply(tf.gather(nn1, indices) , tf.reshape(g, (6, 1))) , _nn1_samples)
        nn1 = tf.tensor_scatter_nd_update(nn1,tf.expand_dims(indices, 1),_nn1_samples)

        _nn2_sample = tf.math.add_n([tf.math.reduce_sum(tf.math.multiply(_vRel , tf.reshape(g1, (g1.shape[0], 1))), axis=0, keepdims=False)  , _vRel, tf.math.reduce_sum(_nn1_samples, axis=0, keepdims=False)])
        indices = tf.constant(_relation_index)
        nn2 = tf.tensor_scatter_nd_update(nn2,tf.expand_dims([indices], 1), tf.reshape(_nn2_sample,(1, _nn2_sample.shape[0])))
  
        indices = tf.constant(_head_index)
        _nn0_sample = tf.math.add(tf.math.reduce_sum(tf.math.multiply(_vHead , tf.reshape(g, (g.shape[0], 1))), axis=0, keepdims=False)  , _nn2_sample)
        nn0 = tf.tensor_scatter_nd_update(nn0,tf.expand_dims([indices], 1), tf.reshape(_nn0_sample,(1, _nn0_sample.shape[0])))
    print(time.time() - start)        
    # if x % 10 == 0 and x > 20:
# _mrrList = []
# # for triple in validation:
# _mrrList.append(calculateRank(validation[0,:]))
# print('total one sample calc time =>  ' , time.time() - start) 
# 1696.807552099228

In [98]:
start = time.time()
_mrrList = []
triple = tf.gather(validation,1200)
_mrrList.append(calculateRank(triple))
print('total one sample calc time =>  ' , time.time() - start) 
print(_mrrList)

tf.Tensor([[14920]], shape=(1, 1), dtype=int64) tf.Tensor([[14941]], shape=(1, 1), dtype=int64)
total one sample calc time =>   1.0491907596588135
[<tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[14920]])>]


In [95]:
tf.gather(train,22091)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([12052,   603,   184], dtype=int32)>

In [ ]:
t = tf.constant([ [1,2,5],[1,2,6], [3,3,7], [2, 5, 4],[0,3,3]])
e = tf.constant([0,1,2,3,4,5,6,7])
# trip = tf.constant([1,2,2])
# indexes = tf.where(tf.math.logical_and( (tf.gather(t, 0, axis=1) == 1) , (tf.gather(t, 1, axis=1) == 2)))
# indexes = tf.reshape(indexes, (indexes.shape[0] ))
# existEntites = tf.gather(tf.gather(t, indexes), 2, axis=1)
# # existEntites = tf.reshape(existEntites ,  (existEntites.shape[0] , 1) )
# print(e)
# print(existEntites)


# shape = tf.constant([e.shape[0]])
# scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), existEntites, shape)
# scatter = tf.subtract(e , scatter)
# print(tf.where(tf.math.greater(scatter, 0)) )


# TODO : tf.tensor_scatter_nd_update(tensor, indices, updates) test
h = tf.fill([e.shape[0], 1], 1)
r = tf.reshape(tf.fill(e.shape, 2) ,  (tf.fill(e.shape, 2).shape[0] , 1))
e = tf.reshape(e , (e.shape[0], 1))
print(h)
couraptedH =tf.concat([  h, r, e],1)
couraptedH
# tf.tensor_scatter_nd_update(couraptedH, indices, updates)
# couraptedH[:,0] =selectedEntities
# couraptedH[:,1] =_tail_index
# couraptedH[:,2] =_relation_index

a = tf.constant([34, 56, 12])
b = tf.constant([56, 55])
c = tf.constant([57])

intersection = tf.sets.intersection(tf.reshape(a, (1, a.shape[0])),tf.reshape(b, (1, b.shape[0])),tf.reshape(c, (1, c.shape[0])))
print(intersection.values)



In [22]:
mat =tf.constant( [1,
       3,
       2,
       8,
       7,
       4,
       5])
sortInd = tf.math.top_k( mat ,k = mat.shape[0] - 1).indices
print(sortInd)
tf.where(tf.math.top_k( mat ,k = mat.shape[0] - 1).indices  == tf.gather(mat,mat.shape[0] - 1 ))

tf.Tensor([3 4 6 5 1 2], shape=(6,), dtype=int32)


<tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[3]])>